In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
with open('data.json', 'r', encoding = 'utf-8') as file:
    news = json.load(file)

In [3]:
rows = [(value, key) for key, values in news.items() for value in values]

news_df = pd.DataFrame(rows, columns=["News", "Category"])

In [4]:
news_df

,News,Category
0,"«ՀայաՔվե» ազգային քաղաքացիական միավորումը, խստ...",Քաղաքական
1,"Տպավորություն էր, ԱԱԾ-ականներն ամաչում էին, գի...",Քաղաքական
2,Այսպես կոչված Զանգեզուրի միջանցքի տրամաբանությ...,Քաղաքական
3,"Նա, հղում կատարելով Հայաստանի մասին ԵԽԽՎ զեկու...",Քաղաքական
4,Բոլորն էլ կարող են հասնել Մայր Աթոռ ու ցույց տ...,Քաղաքական
...,...,...
523,Թել-Ավիվի համալսարանի աշխատակից Դանիելա Յակուբ...,Առոջապահական
524,Այրվածքաբանության ազգային կենտրոնում գործող ճա...,Առոջապահական
525,"Մսի ոսկորների արգանակը պարունակում է կոլագեն, ...",Առոջապահական
526,"«Եթե մարդիկ իրականում արևից դառնային թմրամոլ, ...",Առոջապահական


In [5]:
import random

In [6]:
News_data = news_df.News

In [7]:
Category_data = news_df.Category

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
N_train, N_test, C_train, C_test = train_test_split(News_data, Category_data, test_size=0.125, random_state=42)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer()

In [12]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
model = RandomForestClassifier()

In [15]:
params = {
    'model__n_estimators': [60, 100],
    'model__max_depth': [2, 4, 8],
    'model__max_features': ['sqrt', 'log2'],
    'model__class_weight': ['balanced', None],
    'model__criterion': ['gini', 'entropy', 'log_loss']
}

In [16]:
from sklearn.pipeline import Pipeline

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
pipe = Pipeline([('tfidf', tfidf), ('model', model)])

In [19]:
final_model = GridSearchCV(pipe, param_grid=params, cv=3, verbose=1)

In [20]:
final_model.fit(N_train, C_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', RandomForestClassifier())]),
             param_grid={'model__class_weight': ['balanced', None],
                         'model__criterion': ['gini', 'entropy', 'log_loss'],
                         'model__max_depth': [2, 4, 8],
                         'model__max_features': ['sqrt', 'log2'],
                         'model__n_estimators': [60, 100]},
             verbose=1)

In [21]:
pred = final_model.predict(N_test)

In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(pred, C_test))

              precision    recall  f1-score   support

Առոջապահական       1.00      1.00      1.00        17
     Կրթական       0.81      1.00      0.90        13
 Հասարակական       1.00      0.72      0.84        18
   Քաղաքական       0.90      1.00      0.95        18

    accuracy                           0.92        66
   macro avg       0.93      0.93      0.92        66
weighted avg       0.94      0.92      0.92        66



In [24]:
import pickle

In [35]:
with open('model.pkl', 'wb') as file:
    pickle.dump(final_model, file)